In [24]:
import os
import pandas as pd
import numpy as np
import pandas_datareader
from pandas_datareader import data as pdr
from datetime import datetime, timedelta
import yfinance as yf

def stockData(sList, stDate, endDate):
    """Take list of stocks and pull data for that stock and create technical indicator variables
    sList requires list of stocks, dates should be entered in format 'YYYY-MM-DD' """
   
    data = []
    for element in sList:
        try:
            temp = pdr.get_data_yahoo(element, start = stDate, end = endDate)
            #add an indentifier
            temp['Symbol'] = element
            temp['Close'] = temp['Close']
        except: 
            #If stock symbol cannot be found make it $1
            print (element)
            temp['Symbol'] = element
            temp['Close'] = 1
        else:
        #add data to list 
            data.append(temp)
        #convert to dataframe    
        df = pd.concat(data)
        #drop rows for which we will not have all data points calculated
        df = df.dropna()
        #dates above what user entered
        df = df.loc[stDate : endDate]
        #order and limit columns to those needed
        df = df.loc[:, ['Symbol', 'Close']]
        df['Day'] = pd.to_datetime(df.index)

    
    return(df)


In [25]:
#set working directory 
os.chdir("/Users/bkrei/Desktop/Bk's Stuff Desktop/School/Github NEW PATH/Portfolio-Challenge")

In [42]:
#Read in individual portfolios
portfolios = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/New%20Baseline.csv")

In [28]:
port1 = portfolios.loc[portfolios['Port_Num'] == 1]

In [43]:
port2 = portfolios.loc[portfolios['Port_Num'] == 2]

In [30]:
#List of all ticker symbols
symbols1 = port1['Symbol'].drop_duplicates()
symbols1.head(10)

0     AAPL
1    AGTHX
2     AMZN
3      CHE
4     DOCU
5    GSGRX
6     ICLN
7    OEGAX
8     PBFS
9      QQQ
Name: Symbol, dtype: object

In [44]:
#List of all ticker symbols
symbols2 = port2['Symbol'].drop_duplicates()
symbols2.head(10)

283     AAPL
284      IBM
285      PIR
286      WMT
287    GOOGL
288     DOCU
289      GCO
290     ATLC
291      XOM
292      FCA
Name: Symbol, dtype: object

In [39]:
symbols1.to_csv("symbols port1.csv", header = False)
symbols2.to_csv("symbols port2.csv", header = False)

In [40]:
#Prices Port 1
#Pass the ticker list to the function, print invalid stock symbols and create dataframe with intial prices
beg_prices1 = stockData(list(symbols1), '2019-09-06', '2019-09-06')
beg_prices1.shape

CASHPIO


(160, 3)

In [41]:
beg_prices1.to_csv("prices port1.csv", header = False)

In [35]:
#Error handling port 1
#er1 = stockData(['SNAP'], '2019-09-06', '2019-09-06')

In [45]:
#Prices Port 2
#Pass the ticker list to the function, print invalid stock symbols and create dataframe with intial prices

beg_prices2 = stockData(list(symbols2), '2019-09-27', '2019-09-27')
beg_prices2.shape

(104, 3)

In [8]:
#Pull SNAP which for some reason is the only stock with an issue.
#er2 = stockData(['SNAP'], '2019-09-27', '2019-09-27')

In [46]:
beg_prices2.to_csv("prices port2.csv", header = False)

In [9]:
#Combine the main and SNAP dataframes
#frames = [beg_prices, er]
#beg_prices = pd.concat(frames)

In [4]:
beg_prices = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/beg_prices.csv")
combined = pd.merge(portfolios, beg_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [5]:
combined.head(5)

,Name,Symbol,% of Portfolio,Cost Basis,Date,Close,Day
0,Adrienne Villano,AAPL,0.07,14000,2019-09-06,213.259995,2019-09-06
1,Adrienne Villano,AGTHX,0.11,22000,2019-09-06,50.000000,2019-09-06
2,Adrienne Villano,AMZN,0.07,14000,2019-09-06,1833.510010,2019-09-06
3,Adrienne Villano,CHE,0.18,36000,2019-09-06,439.820007,2019-09-06
4,Adrienne Villano,DOCU,0.05,10000,2019-09-06,56.270000,2019-09-06


In [6]:
#Calc # of shares purchased
combined["# of Shares"] = combined["Cost Basis"]/ combined["Close"]
#Convert names to proper case
#combined['Name'] = list(map(lambda x: x.title(), combined['Name']))
#combined.head()

In [15]:
combined.to_csv("initial portfolios.csv")

In [7]:
initportfolios = pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/initial%20portfolios.csv")

# Simple Comparison (Initial vs Current Value) START HERE TO UPDATE

In [34]:
#Select date to compare to initial prices
cur_prices = stockData(list(symbols), '2019-09-27', '2019-09-27')

TSS
CASHPIO
SKIS


In [46]:
#Pull SNAP which for some reason is the only stock with an issue.
er2 =  stockData(['SNAP', 'TRBCX', 'SHAK'], '2019-09-27', '2019-09-27')
er3 =  stockData(['TSS'], '2019-09-17', '2019-09-17')
er4 =  stockData(['SKIS'], '2019-09-24', '2019-09-24')

In [50]:
#Combine the main and SNAP dataframes
frames2 = [cur_prices, er2, er3, er4]
cur_prices = pd.concat(frames2)

In [51]:
cur_prices.sort_values("Symbol", inplace = True)
cur_prices.drop_duplicates(subset = "Symbol", inplace = True)

In [52]:
cur_prices.to_csv("cur_prices.csv")

In [25]:
#cur_prices= pd.read_csv("https://raw.githubusercontent.com/bkreis84/Portfolio-Challenge/master/cur_prices.csv")
#cur_prices.head()


,Date,Symbol,Close,Day
0,2019-09-27,AAL,27.010000,2019-09-27
1,2019-09-27,AAPL,218.820007,2019-09-27
2,2019-09-27,ACB,4.620000,2019-09-27
3,2019-09-27,ACER,3.300000,2019-09-27
4,2019-09-27,ACST,1.960000,2019-09-27


In [53]:
detail = pd.merge(initportfolios, cur_prices, how = 'left', on = "Symbol", validate="many_to_one")

In [54]:
detail.head()

,Unnamed: 0,Name,Symbol,% of Portfolio,Cost Basis,Date,Close_x,Day_x,# of Shares,Close_y,Day_y
0,0,Adrienne Villano,AAPL,0.07,14000,2019-09-06,213.259995,2019-09-06,65.647568,218.820007,2019-09-27
1,1,Adrienne Villano,AGTHX,0.11,22000,2019-09-06,50.000000,2019-09-06,440.000000,48.970001,2019-09-27
2,2,Adrienne Villano,AMZN,0.07,14000,2019-09-06,1833.510010,2019-09-06,7.635628,1725.449951,2019-09-27
3,3,Adrienne Villano,CHE,0.18,36000,2019-09-06,439.820007,2019-09-06,81.851665,413.179993,2019-09-27
4,4,Adrienne Villano,DOCU,0.05,10000,2019-09-06,56.270000,2019-09-06,177.714589,60.639999,2019-09-27


In [55]:
detail['Current_Value'] = detail['Close_y'] * detail['# of Shares']
detail['Perc_Return'] = detail['Close_y']/detail['Close_x']  -1
detail.head(5)

,Unnamed: 0,Name,Symbol,% of Portfolio,Cost Basis,Date,Close_x,Day_x,# of Shares,Close_y,Day_y,Current_Value,Perc_Return
0,0,Adrienne Villano,AAPL,0.07,14000,2019-09-06,213.259995,2019-09-06,65.647568,218.820007,2019-09-27,14365.001320,0.026072
1,1,Adrienne Villano,AGTHX,0.11,22000,2019-09-06,50.000000,2019-09-06,440.000000,48.970001,2019-09-27,21546.800537,-0.020600
2,2,Adrienne Villano,AMZN,0.07,14000,2019-09-06,1833.510010,2019-09-06,7.635628,1725.449951,2019-09-27,13174.893613,-0.058936
3,3,Adrienne Villano,CHE,0.18,36000,2019-09-06,439.820007,2019-09-06,81.851665,413.179993,2019-09-27,33819.470439,-0.060570
4,4,Adrienne Villano,DOCU,0.05,10000,2019-09-06,56.270000,2019-09-06,177.714589,60.639999,2019-09-27,10776.612564,0.077661


In [56]:
detail.to_csv("pt1-detail.csv")

### Individual Performance

In [29]:
ranking = detail.groupby("Name")['Cost Basis', 'Current_Value'].sum().reset_index().sort_values("Current_Value", ascending=False)

In [30]:
ranking['Perc_Return'] = ranking['Current_Value']/ranking['Cost Basis'] -1
ranking['Rank'] = ranking['Perc_Return'].rank(ascending=False)
ranking.head(25)

,Name,Cost Basis,Current_Value,Perc_Return,Rank
12,Colton Young,200000,227388.447251,0.136942,1.0
26,Joelean Mcbride,200000,222726.630388,0.113633,2.0
47,Robert Kovacs,200000,216018.302691,0.080092,3.0
29,Kelly Nicholson,200000,204566.838918,0.022834,4.0
37,Marie Barkamian,200000,204011.835997,0.020059,5.0
19,Elise Shaver,200000,203794.723531,0.018974,6.0
40,Nick Capullo,200000,202379.152299,0.011896,7.0
21,Estelle Genest,200000,202310.237055,0.011551,8.0
27,Joseph Lindner,200000,200052.791523,0.000264,9.0
45,Randy Benedict,200000,199943.841909,-0.000281,10.0


In [31]:
ranking.to_csv("individual rankings.csv")

### Best Individual Stock Returns

In [32]:
best_stocks = detail.drop_duplicates(['Symbol']).reset_index().sort_values(['Perc_Return'], ascending=False)

In [33]:
best_stocks = best_stocks.loc[:, ['Symbol', 'Perc_Return']]
best_stocks.head(10)

,Symbol,Perc_Return
79,PIR,0.764610
165,HOV,0.451537
171,HOME,0.291437
234,RVNC,0.265869
106,GME,0.238979
203,FNMAL,0.209600
108,UEPS,0.197452
207,ACER,0.174377
41,CPRI,0.162095
148,ATLC,0.150065


In [34]:
best_stocks.to_csv("stock ranking.csv")

# Trend

In [ ]:
#Pull Only necessary columns
combined = combined.loc[:, ['Name', 'Symbol', '# of Shares']]

In [ ]:
Price_History = stockData(stocks['Ticker'], '2019-08-01', '2019-09-06')

In [ ]:
historical = pd.merge(combined, Price_History, how = 'outer', on = "Symbol", validate="many_to_many")


In [ ]:
historical['Value'] = historical['# of Shares'] * historical['Close']
historical.head()

In [ ]:
trend_data = historical.groupby(['Name', 'Day'])['Value'].sum().reset_index().sort_values(['Name', 'Day'])

In [ ]:
trend_data.head()

In [1]:
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode